In [ ]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from hyperparameters import hyperparameters
import matplotlib as mpl

from IPython.display import display
plt.style.use('bmh')
mpl.rcParams.update({
    "grid.linestyle" : "dashed",
    "axes.facecolor" : "white",
    "axes.spines.top" : False,
    "axes.spines.right" : False,
    "legend.frameon" : False,
    "figure.figsize" : (8, 5),
    "figure.dpi" : 300,
})
%matplotlib inline

# Suppress sklearn deprecated warnings
import warnings
def warn(*args, **kwargs): pass
warnings.warn = warn
np.set_printoptions(threshold=sys.maxsize)

np.random.seed(42)

In [ ]:
# TODO make test for dataset with creatina column
# Dataset without thyroid = 18 features (including survive7y)
# Dataset with thyroid = 27 features (including survive7y)
# With columns that have missing values, 23 and 32
# Default 18
n_features = 18
extra_path = n_features != 27 and n_features != 18
dropped_na_key = "dropped_na/"
mean_key = "mean/"
path = f"data/{n_features}features/{mean_key if extra_path else '' }"
path_models = f"models/{n_features}features/{mean_key if extra_path else '' }"
output_models = f"models_output/{n_features}features/{mean_key if extra_path else '' }"
print(path_models)
print(path)

In [ ]:
# Read data
df_train = pd.read_csv(f"{path}train.csv", index_col=0)
df_valid = pd.read_csv(f"{path}valid.csv", index_col=0)
df_test = pd.read_csv(f"{path}test.csv", index_col=0)
print(len(df_train) + len(df_valid) + len(df_test))
print(len(df_train.columns))


train, valid, test = df_train.to_numpy(), df_valid.to_numpy(), df_test.to_numpy()

# y_**** contains the value of Survive7y as a list
# X_**** contains everything except for Survive7y as a list of list
X_train, y_train = train[:, :-1], train[:, -1]
X_valid, y_valid = valid[:, :-1], valid[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]
feat_names = list(df_train.columns)
# Print how Survive7y are distribuited in each set
from collections import Counter
print(Counter(y_train))
print(Counter(y_valid))
print(Counter(y_test))

# All the numerical features that can be standardized
from utils import get_preprocess_std_num
preprocess_std = get_preprocess_std_num(feat_names)


# Preprocessed ready-to-use train and valid set
process_tmp = preprocess_std.fit(X_train)
X_train_std = process_tmp.transform(X_train)
X_valid_std = process_tmp.transform(X_valid)

#If you want to print the resulting df
#df_scaled = pd.DataFrame(X_train_std,columns = preprocess_std.get_feature_names_out())
#display(df_scaled)

### Training


In [ ]:
from functools import partial
from train import report, evaluate, train_and_evaluate

train_partial = partial(
    train_and_evaluate, 
    preprocess_std, 
    X_train=X_train,
    y_train=y_train,
    X_valid=X_valid,
    y_valid=y_valid,
    scoring="f1_macro", 
    iter=5000, 
    save=True,
    path_models = path_models,
    output_models = output_models
)

In [ ]:
from sklearn.linear_model import LogisticRegression

hyperparams = hyperparameters["lr"] 
#Default is None (thus weight = 1). Balanced uses the formula n_samples / (n_classes * np.bincount(y))
model = LogisticRegression(class_weight="balanced")
train_partial(model=model, hyperparams=hyperparams, savename="lr")
# metrics.plot_roc_curve(pipe, X_valid, y_valid)

# import math
# w = logreg.coef_[0]
# feature_importance = pd.DataFrame(df_feat.columns[:-1], columns=["features"])
# feature_importance["importance"] = pow(math.e, w)
# feature_importance = feature_importance.sort_values(by = ["importance"], ascending=False)
# feature_importance

In [ ]:
from sklearn.svm import SVC
hyperparams = hyperparameters["svc"] 

model = SVC(class_weight="balanced", probability=True)
train_partial(model=model, hyperparams=hyperparams, savename="svc")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

hyperparams = hyperparameters["knn"] 

model = KNeighborsClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="knn")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

hyperparams = hyperparameters["rf"] 

model = RandomForestClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="rf")

# feature importance use permutation importance
# importance = rf_rand.best_estimator_["model"].feature_importances_
# plt.bar(list(range(len(importance))), importance)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

hyperparams = hyperparameters["adaboost"] 

model = AdaBoostClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="adaboost")

In [ ]:
from sklearn.neural_network import MLPClassifier
import random

hyperparams = hyperparameters["nn"] 

model = MLPClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="nn")

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

hyperparams = hyperparameters["gb"] 

model = GradientBoostingClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="gb")

In [ ]:
#Don't run this in jupyter within vscode, run this with notebooks within browsers.
import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'

import xgboost as xgb

hyperparams = hyperparameters["xgb"] 

model = xgb.XGBClassifier(n_jobs=1)
train_partial(model=model, hyperparams=hyperparams, savename="xgb")